# Variance Replicate Tables for the 2015 ACS

* [Documentation](https://www.census.gov/programs-surveys/acs/technical-documentation/variance-tables.html)
* [Data Files](https://www2.census.gov/programs-surveys/acs/replicate_estimates/2015/data/5-year/)


In [8]:
import metatab as mt
import pandasreporter as pr
import pandas as pd
import numpy as np
import geoid

doc = mt.open_package('../_packages/census.gov-varrep_tables_support-2011e2015-1/')
doc

In [128]:
B01003 = pr.get_varrep_dataframe(2015,  'B01003', '140' , state='06', cache=True)
B01003 = B01003[pd.notnull(B01003['GEOID'])]
B01003.head()

,TBLID,GEOID,NAME,ORDER,TITLE,estimate,moe,CME,SE,Var_Rep1,...,Var_Rep71,Var_Rep72,Var_Rep73,Var_Rep74,Var_Rep75,Var_Rep76,Var_Rep77,Var_Rep78,Var_Rep79,Var_Rep80
2,B01003,14000US06001400100,"Census Tract 4001, Alameda County, California",1.0,Total,2952.0,186.0,+/-186,113.0,3006.0,...,3054.0,2824.0,2974.0,2987.0,2916.0,2888.0,2880.0,2977.0,2925.0,2872.0
3,B01003,14000US06001400200,"Census Tract 4002, Alameda County, California",1.0,Total,1984.0,99.0,+/-99,60.0,1949.0,...,1998.0,1957.0,1993.0,1995.0,1993.0,2001.0,2029.0,1989.0,1978.0,1971.0
4,B01003,14000US06001400300,"Census Tract 4003, Alameda County, California",1.0,Total,5377.0,524.0,+/-524,319.0,5442.0,...,5429.0,5411.0,5449.0,5632.0,5282.0,5281.0,5686.0,5495.0,5187.0,5170.0
5,B01003,14000US06001400400,"Census Tract 4004, Alameda County, California",1.0,Total,4105.0,305.0,+/-305,185.0,4223.0,...,4181.0,4119.0,4172.0,3983.0,4017.0,4120.0,4156.0,4027.0,4183.0,4169.0
6,B01003,14000US06001400500,"Census Tract 4005, Alameda County, California",1.0,Total,3651.0,299.0,+/-299,182.0,3650.0,...,3650.0,3634.0,3634.0,3657.0,3672.0,3707.0,3594.0,3765.0,3609.0,3657.0


In [129]:
# Break out the geoid by county
def parse_geoid(tv):
    from geoid.core import parse_to_gvid
    tract = parse_to_gvid(tv)
    
    return pd.Series({'state_fips':tract.state, 
                      'county_fips':tract.county, 
                      'tract_fips':tract.tract})

B01003 = B01003.merge(B01003.GEOID.apply(parse_geoid), left_index=True, right_index=True) 
B01003.head()

,TBLID,GEOID,NAME,ORDER,TITLE,estimate,moe,CME,SE,Var_Rep1,...,Var_Rep74,Var_Rep75,Var_Rep76,Var_Rep77,Var_Rep78,Var_Rep79,Var_Rep80,county_fips,state_fips,tract_fips
2,B01003,14000US06001400100,"Census Tract 4001, Alameda County, California",1.0,Total,2952.0,186.0,+/-186,113.0,3006.0,...,2987.0,2916.0,2888.0,2880.0,2977.0,2925.0,2872.0,1,6,400100
3,B01003,14000US06001400200,"Census Tract 4002, Alameda County, California",1.0,Total,1984.0,99.0,+/-99,60.0,1949.0,...,1995.0,1993.0,2001.0,2029.0,1989.0,1978.0,1971.0,1,6,400200
4,B01003,14000US06001400300,"Census Tract 4003, Alameda County, California",1.0,Total,5377.0,524.0,+/-524,319.0,5442.0,...,5632.0,5282.0,5281.0,5686.0,5495.0,5187.0,5170.0,1,6,400300
5,B01003,14000US06001400400,"Census Tract 4004, Alameda County, California",1.0,Total,4105.0,305.0,+/-305,185.0,4223.0,...,3983.0,4017.0,4120.0,4156.0,4027.0,4183.0,4169.0,1,6,400400
6,B01003,14000US06001400500,"Census Tract 4005, Alameda County, California",1.0,Total,3651.0,299.0,+/-299,182.0,3650.0,...,3657.0,3672.0,3707.0,3594.0,3765.0,3609.0,3657.0,1,6,400500


In [122]:

def varrep_se(x):
    """Compute the standard error from a set of variance replicates"""
    return int(round(np.sqrt((4/80) * sum((x.loc['estimate']-x.loc[varcols])**2)),0))

df2['est_calc'] = np.round(np.mean(df2[varcols],axis=1)).astype(int)
df2['se_calc'] = df2.apply(f, axis=1)


In [132]:
B01003.groupby('county_fips').sum().sort_values('estimate', ascending = False)

,ORDER,estimate,moe,SE,Var_Rep1,Var_Rep2,Var_Rep3,Var_Rep4,Var_Rep5,Var_Rep6,...,Var_Rep73,Var_Rep74,Var_Rep75,Var_Rep76,Var_Rep77,Var_Rep78,Var_Rep79,Var_Rep80,state_fips,tract_fips
county_fips,,,,,,,,,,,,,,,,,,,,,
37,2346.0,10038388.0,886694.0,538994.0,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,...,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,10038388.0,14076,957321337
73,628.0,3223096.0,279982.0,170218.0,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,...,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,3223096.0,3768,8863592
59,583.0,3116069.0,239330.0,145499.0,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,...,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,3116069.0,3498,39380550
65,453.0,2298032.0,213637.0,129869.0,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,...,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,2298032.0,2718,32910046
71,369.0,2094769.0,197725.0,120199.0,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,...,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,2094769.0,2214,4937688
85,372.0,1868149.0,140660.0,85517.0,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,...,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,1868149.0,2232,188482024
1,361.0,1584983.0,126026.0,76612.0,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,...,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,1584983.0,2166,156424046
67,317.0,1465832.0,126726.0,77035.0,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,...,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,1465832.0,1902,3163344
13,208.0,1096068.0,82825.0,50345.0,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,...,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,1096068.0,1248,71711231


In [134]:
import pandasreporter as pr

api = pr.CensusApi()
ds = api.get_dataset('ACSSF5Y2015')
df = ds.fetch_dataframe( 'GEOID', 'NAME', 'B01003_001E', 'B01003_001M', geo_in='state:06', geo_for='county:*')
df.head()

,GEOID,NAME,B01003_001E,B01003_001M,state,county
0,05000US06001,"Alameda County, California",1584983,0,06,001
1,05000US06003,"Alpine County, California",1131,167,06,003
2,05000US06005,"Amador County, California",36995,0,06,005
3,05000US06007,"Butte County, California",222564,0,06,007
4,05000US06009,"Calaveras County, California",44767,0,06,009


In [113]:
df2 = df.drop(['TBLID','NAME','CME', 'GEOID','ORDER','TITLE'],1)
varcols = [ 'Var_Rep'+str(i) for i in range(1,81)]

In [114]:
df2.head()

,estimate,moe,SE,Var_Rep1,Var_Rep2,Var_Rep3,Var_Rep4,Var_Rep5,Var_Rep6,Var_Rep7,...,Var_Rep71,Var_Rep72,Var_Rep73,Var_Rep74,Var_Rep75,Var_Rep76,Var_Rep77,Var_Rep78,Var_Rep79,Var_Rep80
0,5209,403,245,5311,4897,5089,5285,5313,5179,5141,...,5218,5099,5292,5185,5156,5176,5295,5214,5171,5117
1,2388,319,194,2423,2311,2510,2343,2476,2524,2382,...,2439,2347,2409,2282,2373,2399,2378,2355,2451,2269
2,207,101,61,155,174,182,237,234,206,146,...,256,220,158,217,242,175,222,191,229,205
3,59,56,34,45,72,49,59,38,36,43,...,64,70,40,67,80,58,67,68,39,71
4,86,65,40,83,71,99,84,90,74,75,...,68,64,120,64,106,78,82,100,59,93


# Example calculations 

The first way to test SE calculations using variance replicates is to reproduce the estimate and SE for each row.  


In [115]:

            

c = df2[['estimate','est_calc','SE','se_calc']].copy()

In [117]:

[['GEOID', 'state_fips', 'county_fips', 'tract_fips']]

,GEOID,state_fips,county_fips,tract_fips
0,14000US11001000100,11,1,100
1,14000US11001000100,11,1,100
2,14000US11001000100,11,1,100
3,14000US11001000100,11,1,100
4,14000US11001000100,11,1,100
5,14000US11001000100,11,1,100
6,14000US11001000100,11,1,100
7,14000US11001000100,11,1,100
8,14000US11001000100,11,1,100
9,14000US11001000100,11,1,100


In [88]:

c['est_diff'] = (((c['est_calc'] - c['estimate']) / c['estimate']) * 100).fillna(0) 
c['est_eq'] = np.abs(c['est_diff']) < 2
c[c.estimate != 0].head()

,estimate,est_calc,SE,se_calc,est_diff,est_eq
0,5209,5200,245,245,-0.172778,True
1,2388,2383,194,194,-0.209380,True
2,207,206,61,61,-0.483092,True
3,59,58,34,34,-1.694915,True
4,86,87,40,40,1.162791,True


In [89]:
c['se_eq'] = c.SE == c.se_calc
c[c.estimate == 0].head()

,estimate,est_calc,SE,se_calc,est_diff,est_eq,se_eq
6,0,0,10,0,0.0,True,False
31,0,0,10,0,0.0,True,False
51,0,0,7,0,0.0,True,False
52,0,0,7,0,0.0,True,False
53,0,0,7,0,0.0,True,False


In [82]:
df[df.estimate == 0].head()

,TBLID,GEOID,NAME,ORDER,TITLE,estimate,moe,CME,SE,Var_Rep1,...,Var_Rep71,Var_Rep72,Var_Rep73,Var_Rep74,Var_Rep75,Var_Rep76,Var_Rep77,Var_Rep78,Var_Rep79,Var_Rep80
6,B01001,14000US11001000100,"Census Tract 1, District of Columbia, District...",7,18 and 19 years,0,17,+/-17,10,0,...,0,0,0,0,0,0,0,0,0,0
31,B01001,14000US11001000100,"Census Tract 1, District of Columbia, District...",32,20 years,0,17,+/-17,10,0,...,0,0,0,0,0,0,0,0,0,0
51,B01001,14000US11001000201,"Census Tract 2.01, District of Columbia, Distr...",3,Under 5 years,0,12,+/-12,7,0,...,0,0,0,0,0,0,0,0,0,0
52,B01001,14000US11001000201,"Census Tract 2.01, District of Columbia, Distr...",4,5 to 9 years,0,12,+/-12,7,0,...,0,0,0,0,0,0,0,0,0,0
53,B01001,14000US11001000201,"Census Tract 2.01, District of Columbia, Distr...",5,10 to 14 years,0,12,+/-12,7,0,...,0,0,0,0,0,0,0,0,0,0
